In [ ]:
from nuscenes.nuscenes import NuScenes
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib as mpl
import os
import cv2
import PIL
import numpy as np
from datetime import datetime
from celluloid import Camera
from pyquaternion import Quaternion
from nuscenes.utils.data_classes import LidarPointCloud

mpl.rc('figure', max_open_warning = 0)
%matplotlib inline

In [ ]:
nusc = NuScenes(version='v1.0-mini', dataroot='./data/nusc', verbose=True)

# 1a)

In [ ]:
my_sample = nusc.sample[0]
sample_data = nusc.get('sample_data', my_sample['data']['LIDAR_TOP'])
pcl_path = sample_data['filename']
nusc.render_sample_data(sample_data['token'])
pc =LidarPointCloud.from_file(os.path.join('./data/nusc',pcl_path))
color = pc.points[3] / np.max(pc.points[3])
plt.scatter(pc.points[0], pc.points[1], c=color, alpha=0.2, s=0.3)

In [ ]:
X = pc.points[0:2].T
kmeans = KMeans(n_clusters=69)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=0.2, cmap='viridis')
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=40, alpha=0.5);

# 1b)

In [ ]:
def get_lidar_pointcloud(sample_token):
    sample_record = nusc.get('sample', sample_token)
    pointsensor_channel = 'LIDAR_TOP'
    pointsensor_token = sample_record['data'][pointsensor_channel]
    pointsensor = nusc.get('sample_data', pointsensor_token)
    pcl_path = nusc.get('sample_data', pointsensor_token)['filename']
    pc =LidarPointCloud.from_file(os.path.join('./data/nusc',pcl_path))
    return pointsensor_token, pc.points
def rotate_pointcloud(cloud, quaternion):
    def rot(an_array):
        return quaternion.rotate(an_array)
    return np.array(list(map(rot, cloud)))
def visualize_absolute(ax, absolute_points, ego_pose, van_translation, relativeboxvectors):
    axes = plt.gca()
    return ax.scatter(absolute_points[0], absolute_points[1], c='red', s=0.3), ax.scatter(ego_pose['translation'][0], ego_pose['translation'][1]),ax.plot([van_translation[0] + relativeboxvector[0] for relativeboxvector in relativeboxvectors], [van_translation[1] + relativeboxvector[1] for relativeboxvector in relativeboxvectors])[0]
def create_bounding_box(annotation_token):
    '''
    creates bounding box used for plotting
    '''
    van_annotation = nusc.get('sample_annotation', annotation_token)
    relativeboxvectors = [(van_annotation['size'][1]/2,van_annotation['size'][0]/2, 0),
                         (-van_annotation['size'][1]/2,van_annotation['size'][0]/2, 0),
                         (-van_annotation['size'][1]/2,-van_annotation['size'][0]/2, 0),
                         (van_annotation['size'][1]/2,-van_annotation['size'][0]/2, 0),
                         (van_annotation['size'][1]/2,van_annotation['size'][0]/2, 0)]
    return np.array(relativeboxvectors), van_annotation['translation']
cat_ls = ['vehicle.bicycle', 'vehicle.bus.bendy', 'vehicle.bus.rigid', 'vehicle.car', 'vehicle.construction'
         'vehicle.motorcycle', 'vehicle.trailer', 'vehicle.truck']

In [ ]:
def clusterSample(sample):
    ann_tokens = sample['anns']
    X_data = np.array([])
    y_data = np.array([])
    valid_ann_num = 0
    for i,current_van_annotation_token in enumerate(ann_tokens):
        van_annotation = nusc.get('sample_annotation', current_van_annotation_token)
        # Only track vehicles
        if van_annotation['category_name'] not in cat_ls:
            continue
        pointsensor_token, points = get_lidar_pointcloud(sample['token'])
        calibrated_sensor_token = nusc.get('sample_data', pointsensor_token)['calibrated_sensor_token']
        calibrated_sensor = nusc.get('calibrated_sensor', calibrated_sensor_token)
        ego_pose = nusc.get('ego_pose', nusc.get('sample_data', pointsensor_token)['ego_pose_token'])

        ### Rotate points around sensor and ego rotation
        pointsensor_quaternion = Quaternion(calibrated_sensor['rotation'])
        ego_quaternion = Quaternion(ego_pose['rotation'])

        sensor_rotated_points = np.dot(pointsensor_quaternion.rotation_matrix, points[:3,:])
        rotated_points = np.dot(ego_quaternion.rotation_matrix, sensor_rotated_points)

        # use broadcasting to add translation to x and y dimension
        absolute_points = rotated_points+np.array(ego_pose['translation'][:3]).reshape(-1,1)

        ### rotate points by inverse bounding box rotation (translation to its reference frame unnecessary for only filtering)
        reverse_van_quaternion = Quaternion(w=-1*van_annotation['rotation'][0], x=van_annotation['rotation'][1], y=van_annotation['rotation'][2], z=van_annotation['rotation'][3])
        points_bounding_rotation = np.dot(reverse_van_quaternion.rotation_matrix, absolute_points)

        #new stuff
        relativeboxvectors, position = create_bounding_box(current_van_annotation_token)
        rotated_box = relativeboxvectors+reverse_van_quaternion.rotate(position)
        x_max = rotated_box[:,0].max()
        x_min = rotated_box[:,0].min()
        y_max = rotated_box[:,1].max()
        y_min = rotated_box[:,1].min()

        ## Remove points that are outside the bounding box
        mask = np.ones(points_bounding_rotation.shape[1], dtype=bool)
        mask = np.logical_and(mask, points_bounding_rotation[0] > x_min)
        mask = np.logical_and(mask, points_bounding_rotation[0] < x_max)
        mask = np.logical_and(mask, points_bounding_rotation[1] > y_min)
        mask = np.logical_and(mask, points_bounding_rotation[1] < y_max)
        points_filtered_rotated = points_bounding_rotation[:,mask]

        ### rotate points back
        van_quaternion = Quaternion(van_annotation['rotation'])
        points_filtered_absolute = np.dot(van_quaternion.rotation_matrix, points_filtered_rotated)
        if (points_filtered_absolute.size != 0):
            valid_ann_num = valid_ann_num + 1
            for px in points_filtered_absolute[0]:
                X_data = np.append(X_data, px)
            for py in points_filtered_absolute[1]:
                y_data = np.append(y_data, py)
    X_data = np.reshape(X_data, (X_data.shape[0], 1))
    y_data = np.reshape(y_data, (y_data.shape[0], 1))
    data = np.append(X_data, y_data, axis=1)
    # Implement K-means
    kmeans = KMeans(n_clusters=valid_ann_num)
    kmeans.fit(data)
    y_kmeans = kmeans.predict(data)
    centers = kmeans.cluster_centers_
    # Implement DBSCAN
    clustering = DBSCAN(eps=3, min_samples=2).fit(data)
    labels = clustering.labels_
    return data, centers, y_kmeans, labels

# Plot
data, centers, y_kmeans, labels = clusterSample(my_sample)
plt.scatter(data[:, 0], data[:, 1], c=y_kmeans, s=1, cmap='viridis')
plt.title('Origin')
plt.show()
plt.scatter(data[:, 0], data[:, 1], c=y_kmeans, s=1, cmap='viridis')

plt.scatter(centers[:, 0], centers[:, 1], c='black', s=40, alpha=0.6)
plt.title('With K-Means')
plt.show()

plt.scatter(data[:, 0], data[:, 1], c=labels, s=1, cmap='viridis')
plt.title('With DBSCAN')
plt.show()

# 1c)

In [ ]:
my_scene = nusc.scene[0]
sample = nusc.get('sample', my_scene['first_sample_token'])
sample_ls = []
sample_ls.append(sample)
while(sample['next'] != ''):
    sample = nusc.get('sample', sample['next'])
    sample_ls.append(sample)
    
# Create movie using kmeans
fig = plt.figure()
camera = Camera(plt.figure())

for curr_sample in sample_ls:
    data, centers, y_kmeans, _ = clusterSample(curr_sample)
    plt.scatter(data[:, 0], data[:, 1], c=y_kmeans, s=1, cmap='viridis')
    plt.scatter(centers[:, 0], centers[:, 1], c='black', s=40, alpha=0.6)
    camera.snap()
    
anim = camera.animate(blit=True)
anim.save("kmeans.gif", writer='pillow')

# Create movie using dbscan
fig = plt.figure()
camera = Camera(plt.figure())

for curr_sample in sample_ls:
    data, _, _, labels = clusterSample(curr_sample)
    plt.scatter(data[:, 0], data[:, 1], c=labels, s=1, cmap='viridis')
    camera.snap()
    
anim = camera.animate(blit=True)
anim.save("dbscan.gif", writer='pillow')

# 2a)

In [ ]:
def getPoints(sample):
    ann_tokens = sample['anns']
    points2D = []
    valid_ann_num = 0
    for i,current_van_annotation_token in enumerate(ann_tokens):
        van_annotation = nusc.get('sample_annotation', current_van_annotation_token)
        # Only track vehicles
        if van_annotation['category_name'] not in cat_ls:
            continue
        valid_ann_num = valid_ann_num + 1
        pointsensor_token, points = get_lidar_pointcloud(sample['token'])
        calibrated_sensor_token = nusc.get('sample_data', pointsensor_token)['calibrated_sensor_token']
        calibrated_sensor = nusc.get('calibrated_sensor', calibrated_sensor_token)
        ego_pose = nusc.get('ego_pose', nusc.get('sample_data', pointsensor_token)['ego_pose_token'])

        ### Rotate points around sensor and ego rotation
        pointsensor_quaternion = Quaternion(calibrated_sensor['rotation'])
        ego_quaternion = Quaternion(ego_pose['rotation'])

        sensor_rotated_points = np.dot(pointsensor_quaternion.rotation_matrix, points[:3,:])
        rotated_points = np.dot(ego_quaternion.rotation_matrix, sensor_rotated_points)

        # use broadcasting to add translation to x and y dimension
        absolute_points = rotated_points+np.array(ego_pose['translation'][:3]).reshape(-1,1)

        ### rotate points by inverse bounding box rotation (translation to its reference frame unnecessary for only filtering)
        reverse_van_quaternion = Quaternion(w=-1*van_annotation['rotation'][0], x=van_annotation['rotation'][1], y=van_annotation['rotation'][2], z=van_annotation['rotation'][3])
        points_bounding_rotation = np.dot(reverse_van_quaternion.rotation_matrix, absolute_points)

        #new stuff
        relativeboxvectors, position = create_bounding_box(current_van_annotation_token)
        rotated_box = relativeboxvectors+reverse_van_quaternion.rotate(position)
        x_max = rotated_box[:,0].max()
        x_min = rotated_box[:,0].min()
        y_max = rotated_box[:,1].max()
        y_min = rotated_box[:,1].min()

        ## Remove points that are outside the bounding box
        mask = np.ones(points_bounding_rotation.shape[1], dtype=bool)
        mask = np.logical_and(mask, points_bounding_rotation[0] > x_min)
        mask = np.logical_and(mask, points_bounding_rotation[0] < x_max)
        mask = np.logical_and(mask, points_bounding_rotation[1] > y_min)
        mask = np.logical_and(mask, points_bounding_rotation[1] < y_max)
        
        points_filtered_rotated = points_bounding_rotation[:,mask]
        
        van_quaternion = Quaternion(van_annotation['rotation'])
        points_filtered_absolute = np.dot(van_quaternion.rotation_matrix, points_filtered_rotated)
        if points_filtered_absolute.size != 0:
            points2D.append(points_filtered_absolute[0:2, :])
    return points2D
        
mean_arr_ls = []
time_ls = []
for curr_sample in sample_ls:
    points2D = getPoints(curr_sample)
    time_ls.append(curr_sample['timestamp'])
    mean_arr = np.array([]).reshape(0, 2)
    for i in range(len(points2D)):
        temp = np.mean(points2D[i], axis=1).reshape(1,2)
        if not np.isnan(np.min(temp)):
            mean_arr = np.append(mean_arr, temp, axis=0)
    mean_arr_ls.append(mean_arr)

In [ ]:
C_00 = 1*np.eye(4)
sigma = 0.01
r = 0.01
R = np.array(((r,0),(0,r)))
C_kk = C_00


for s in range(len(mean_arr_ls)-1):
    cost_mat = np.zeros((mean_arr_ls[s].shape[0], mean_arr_ls[s+1].shape[0]))
    for i in range(mean_arr_ls[s].shape[0]):
        for j in range(mean_arr_ls[s+1].shape[0]):
            cost_mat[i][j] = np.linalg.norm(mean_arr_ls[s][i]-mean_arr_ls[s+1][j])
    row_ind, col_ind = linear_sum_assignment(cost_mat)
    # print('cost matrix shape:', cost_mat.shape)
    # print(cost_mat)
    # print('row', row_ind)
    # print('col', col_ind)
    # print('cost sum:', cost_mat[row_ind, col_ind].sum())
    X_curr = mean_arr_ls[s][row_ind][:, 0]
    y_curr = mean_arr_ls[s][row_ind][:, 1]
    X_next = mean_arr_ls[s+1][col_ind][:, 0]
    y_next = mean_arr_ls[s+1][col_ind][:, 1]
    
    if s == 0:
        num_veh = X_curr.shape[0]
        X_data, y_data = X_curr.reshape(num_veh, 1), y_curr.reshape(num_veh, 1)
    if np.minimum(mean_arr_ls[s].shape[0], mean_arr_ls[s+1].shape[0]) < num_veh:
        continue
    X_data_new, y_data_new = np.zeros((num_veh, 1)), np.zeros((num_veh, 1))
    for i in range(num_veh):
        datatime1 = datetime.fromtimestamp(int(time_ls[s])/1e6)
        datatime2 = datetime.fromtimestamp(int(time_ls[s+1])/1e6)
        t = (datatime2 - datatime1).microseconds/1e6
        F = np.identity(4)
        F[0, 2] = t
        F[1, 3] = t
        velocity_x = (X_next[i] - X_curr[i]) / t
        velocity_y = (y_next[i] - y_curr[i]) / t
        x_kk = np.array([X_next[i], y_next[i], velocity_x, velocity_y]).reshape(4, 1)
        x_k1k = F.dot(x_kk)

        term = np.array([0.5*t*t, 0.5*t*t, t, t]).reshape(4, 1)
        Q = term.dot(sigma).dot(term.T)
        C_k1k = F.dot(C_kk).dot(F.T) + Q

        H = np.zeros((2, 4))
        H[0, 0] = 1
        H[1, 1] = 1
        y_k1 = np.array([X_curr[i], y_curr[i]]).reshape(2, 1)

        S = H.dot(C_k1k).dot(H.T) + R
        K = C_k1k.dot(H.T).dot(np.linalg.inv(S))
        x_k1k1 = x_k1k + K.dot(y_k1 - H.dot(x_k1k))
        # update C_kk
        C_kk = C_k1k - K.dot(S).dot(K.T)
        X_data_new[i] = x_k1k1[0]
        y_data_new[i] = x_k1k1[1]
    X_data = np.append(X_data, X_data_new, axis=1)
    y_data = np.append(y_data, y_data_new, axis=1)

In [ ]:
fig = plt.figure()
camera = Camera(plt.figure())

for i in range(X_data.shape[1]):
    plt.scatter(X_data[0, i], y_data[0, i], c='k')
    plt.scatter(X_data[1, i], y_data[1, i], c='b')
    plt.scatter(X_data[2, i], y_data[2, i], c='r')
    plt.scatter(X_data[3, i], y_data[3, i], c='y')
    camera.snap()
anim = camera.animate(blit=True)
anim.save("multiKalman.gif", writer='pillow')
plt.show()

Sometimes the tracker works

In [ ]:
fig = plt.figure()
print("  Example1")
for i in range(3):
    plt.subplot(2, 2, i+1)
    plt.scatter(X_data[0, i], y_data[0, i], c='k')
    plt.scatter(X_data[1, i], y_data[1, i], c='b')
    plt.scatter(X_data[2, i], y_data[2, i], c='r')
    plt.scatter(X_data[3, i], y_data[3, i], c='y')
plt.show()
print("  Example2")
for i in range(11, 16):
    plt.subplot(3, 2, i-10)
    plt.scatter(X_data[0, i], y_data[0, i], c='k')
    plt.scatter(X_data[1, i], y_data[1, i], c='b')
    plt.scatter(X_data[2, i], y_data[2, i], c='r')
    plt.scatter(X_data[3, i], y_data[3, i], c='y')
plt.show()